In [80]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from IPython.display import display

## Series

### 初始化
#### data和index
定义一个Series最基本的两个参数是data和index，分别提供数据和对应的标签。 大部分能够提供一维列表的数据结构都是可以使用的，例如list, tuple, numpy.array, generator。

In [138]:
inputs = [
    ([1,2], ['a','b']),
    ((1,2), ('a','b')),
    ((x for x in [1,2]), (x for x in ['a','b'])),
    (np.array([1,2]),np.array(['a','b']))    
] # 四种不同数据类型，不过包含的数值是一样的

In [140]:
for data, index in inputs:
    print('<<')    
    print('Type of data is ',type(data), ' and type of index is ',type(index))
    
    s = Series(data, index) #定义Series
    
    display(s)
    print('>>')

<<
Type of data is  <class 'list'>  and type of index is  <class 'list'>


a    1
b    2
dtype: int64

>>
<<
Type of data is  <class 'tuple'>  and type of index is  <class 'tuple'>


a    1
b    2
dtype: int64

>>
<<
Type of data is  <class 'generator'>  and type of index is  <class 'generator'>


Series([], dtype: float64)

>>
<<
Type of data is  <class 'numpy.ndarray'>  and type of index is  <class 'numpy.ndarray'>


a    1
b    2
dtype: int64

>>


data部分可以传入scalar数值

In [154]:
s = Series(1,['a','b','c'])
display(s)

a    1
b    1
c    1
dtype: int64

如果用dict或一个Series传入data的话，dict的keys或另一个Series的index就可以作为index使用。这种情况下，可以省略index参数。

In [155]:
d = {'a':1,'b':2}
s = Series(s) #传入dict
display(s)

a    1
b    1
c    1
dtype: int64

In [156]:
s1 = Series([1,2])
s = Series(s1) #传入Series
display(s)

0    1
1    2
dtype: int64

如果这时额外的指定index, 那么最终构造出的Series会以传入的index为准做匹配。 不在传入的inde中的数据会被舍弃，传入的index中存在但是数据中找不到的数据会被用NaN填充。

In [157]:
d = {'a':1,'b':2}
s = Series(d,['a','c'])
display(s) # 'b'对应的数据被舍弃，'c'对应的数据用NaN填充

a    1.0
c    NaN
dtype: float64

#### name

可以给Series传入一个name作为Series的名字。当Series组成DataFrame时,name会成为DataFrame的columns名称。(DataFrame后面会介绍)

In [158]:
d = {'a':1,'b':2}
s = Series(s,index = ['a','c'],name = 'a series')
display(s)
display(s.name)

a    1.0
c    NaN
Name: a series, dtype: float64

'a series'

#### dtype

创建Series的时候可以传入多类型的数据,Series会尝试自动用一个适合所有成员的数据类型作为最终的数据类型。

例如下面的例子里,int类型和float类型放在一起时，所有的成员都被统一成了float。

In [159]:
s = Series([1,1.1])
display(s)

0    1.0
1    1.1
dtype: float64

下面的例子里，pandas则用了最万能的object类型作为Series的结构。这时每个成员保持了原有的数据类型。

In [160]:
s = Series([True,1,1.1,'1.23'])
display(s)

for x in s:
    print(x,type(x))

0    True
1       1
2     1.1
3    1.23
dtype: object

True <class 'bool'>
1 <class 'int'>
1.1 <class 'float'>
1.23 <class 'str'>


如果不满意pandas的自动处理策略，可以通过指定dtype的方式来指定最终的数据类型。

In [151]:
s = Series([True,1,1.1,'1.23'],dtype = float)
display(s)

for x in s:
    print(x,type(x))

0    1.00
1    1.00
2    1.10
3    1.23
dtype: float64

1.0 <class 'numpy.float64'>
1.0 <class 'numpy.float64'>
1.1 <class 'numpy.float64'>
1.23 <class 'numpy.float64'>


### 常见操作

In [162]:
s = Series([1,2,3,4],['a','b','c','d'])
display(s)

a    1
b    2
c    3
d    4
dtype: int64

#### 数值计算

#### 复制

默认情况下,用```=```将一个Series赋值给另一个变量的时候，两个变量指向的是同一个object，修改其中一个，另一个变量也会相应的改变。

In [174]:
s = Series([1,2,3,4],list('abcd'))

s2 = s

print('s and s2 point to the same object?',s2 is s)
s2['a'] = 10

display(s)

s and s2 point to the same object? True


a    10
b     2
c     3
d     4
dtype: int64

如果希望获得一个独立的Series,但是和原来的Series有同样的数值，可以使用.copy()方法。

In [175]:
s = Series([1,2,3,4],list('abcd'))

s2 = s.copy()

print('s and s2 point to the same object?',s2 is s)
s2['a'] = 10

display(s)

s and s2 point to the same object? False


a    1
b    2
c    3
d    4
dtype: int64

#### 修改label

为所有label增加前缀。

In [179]:
display(s.copy().add_prefix('A_'))

A_a    1
A_b    2
A_c    3
A_d    4
dtype: int64

In [181]:
display(s.copy().add_suffix('_A'))

a_A    1
b_A    2
c_A    3
d_A    4
dtype: int64

#### 逻辑计算

用and去连接所有成员并返回结果。

In [190]:
Series([True,True]).all()

True

注意下面的例子,all()返回的是决定所有成员用and连接之后第一个能决定真假值的object,而不保证是True,False。和and一样。

In [193]:
Series([[],1]).all()

[]

同样，也有any()。等价用or去连接所有成员并返回结果。

In [195]:
Series(['a',1]).any()

'a'

#### 添加数据

In [199]:
s2 = Series([5,6],index = ['e','f'])
s3 = Series([7,8],index = ['g','h'])

display(s.append(s2))

a    1
b    2
c    3
d    4
e    5
f    6
dtype: int64

可以同时添加多个Series

In [200]:
display(s.append([s2,s3]))

a    1
b    2
c    3
d    4
e    5
f    6
g    7
h    8
dtype: int64

如果希望添加后的数组不在使用原有的index，可以添加ignore_index=True参数

In [206]:
s2 = Series([5,6],index = ['a','b'])

display(s.append(s2,ignore_index=True))

0    1
1    2
2    3
3    4
4    5
5    6
dtype: int64

如果添加后的Series没有重复的index，可以设置```verify_integrity=True```。这样如果出现了重复的index,就会raise ValueError。

In [212]:
s2 = Series([5,6],index = ['a','b'])

try:
    display(s.append(s2,verify_integrity=True))
except Exception as e:
    print(type(e),e)

<class 'ValueError'> Indexes have overlapping values: ['a', 'b']


#### 遍历

对每一个成员进行遍历,取出数值后，应用传入的函数,并且将结果组成新的Series。

In [214]:
display(s.apply(lambda x:x+10))

a    11
b    12
c    13
d    14
dtype: int64

如果传入的函数需要额外的参数，也可以在apply中传入。 用args传入positional arguments, 在args之后添加其它函数需要的keyword arguments。

In [216]:
def cal(x, add, mul=1):
    return x*mul+add

s.apply(cal, args = [1], mul = 2)

a    3
b    5
c    7
d    9
dtype: int64

#### 搜索

找到最大值对应的label

In [227]:
a = Series([1,2,None],list('abc'))
display(a)

a    1.0
b    2.0
c    NaN
dtype: float64

如果希望数据中含有NaN时，返回NaN，可以设置```skipna = True```

In [229]:
display(a.argmax(skipna=False))

nan

如果要搜索最小值对应的数值，可以使用.argmin()，用法和.argmax()一样

#### 获得排序
如果希望获得Series中每个元素的排序，可以使用argsort方法。NaN对应的排序用-1指代。

In [233]:
a = Series([1,2,None],list('abc'))
a.argsort()

a    0
b    1
c   -1
dtype: int64

#### 匹配

In [ ]:
s.align

#### label是否存在

In [163]:
'a' in s

True